## Create urban built-up and population density gradients for multiple cities

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Point
from shapely import wkt
import plotly.express as px
import os
import gdal
from rasterio.mask import mask
import rasterio
import pylab as pl
from rasterio.warp import calculate_default_transform, reproject, Resampling
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#dat = pd.read_csv("..\\GHSL_NewVersion\\GHSL_UCDB_new.csv")

In [3]:
cities = ['Accra', 'Buenos Aires', 'Paris', 'Chicago', 'Dhaka', 'Shanghai']

In [4]:
def return_point(city):
    #dat = pd.read_csv("..\\GHSL_NewVersion\\GHSL_UCDB_new.csv")
    #df = dat[dat.UC_NM_MN == city]
    
    #point = Point(df.GCPNT_LON.iloc[0], df.GCPNT_LAT.iloc[0])
    
    import osmnx as ox
    
    point = ox.gdf_from_place('Dhaka, Bangladesh')['geometry'].iloc[0]
    
    return point

In [5]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio accepts them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [6]:
def get_buffer_vars(city):
    
    point = return_point(city)
    buff_500 = point.buffer(0.01, 128)
    buff_1000 = point.buffer(0.015, 128)
    dough_1000 = buff_1000.difference(buff_500)
    buff_1500 = point.buffer(0.02, 128)
    dough_1500 = buff_1500.difference(buff_1000)
    buff_2000 = point.buffer(0.025, 128)
    dough_2000 = buff_2000.difference(buff_1500)
    
    return buff_500, buff_1000, buff_1500, buff_2000

In [14]:
def convert_geom_to_shp(shapely_polygon, city, out_name_postfix):
    df = pd.DataFrame(
    {'City': [city],
     'geometry': [wkt.dumps(shapely_polygon)]})
    
    df['geometry'] = df['geometry'].apply(wkt.loads)
    
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    
    path = "M:\Gaurav\GPSUR\Data\Analysis"
    
    gdf.to_file(os.getcwd()+'\\shapefiles\\{a}_{b}.shp'.format(a=city, b=out_name_postfix))
    
    return path+'\\shapefiles\\{a}_{b}.shp'.format(a=city, b=out_name_postfix)

In [22]:
def reproject_raster(source_path, destination_path):

    dst_crs = 'EPSG:4326'
    
    try:
        with rasterio.open(source_path, CHECK_DISK_FREE_SPACE =False) as src:
            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds)
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height
            })

            with rasterio.open(destination_path, 'w',**kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest)
    except rasterio._err.CPLE_FileIOError:
        print('File too big! Cant open')
    except rasterio._base.RasterioIOError:
        print('Raster file empty!')
        

In [17]:
def get_clipped_density(shpfile_path, raster_path):
    """
    Function to clip the population gridded data by polygons from datasets.
    """
    shp = gpd.read_file(shpfile_path)
    shp.crs = {'init':'epsg:4326'}
    shp = shp.to_crs(epsg=4326)
    
    coords = getFeatures(shp)
    
    new = rasterio.open(raster_path)
    
    try:
        out_img, out_transform = mask(dataset=new, shapes=coords, crop=True)
        new_out = out_img[out_img > 0]
        return new_out.sum()/len(new_out)
        
    except ValueError:
        print('No Overlap between raster and polygon.')
        return

In [44]:
def main(city):
    
    point = return_point(city)
    
    path = r"M:\Gaurav\GPSUR\Data\GHSL_builtup_250m"
    
    temp = os.listdir(path)
    
    buff_500, buff_1000, buff_1500, buff_2000 = get_buffer_vars(city)
    
    dic = {'500m':buff_500, '1000m':buff_1000, '1500m':buff_1500, '2000m':buff_2000}
    
    shp_lis = []
    for res in dic.keys():  
        shp_lis.append(convert_geom_to_shp(dic[res], city, res))
    
    sum_dic = {}
    
    for i in [2000, 2014]:
        for j in temp:
            if str(i) in j.split('_'):
                path_ext = path+"\\"+j
                print(path_ext)
                #TIF_lis = [i for i in os.listdir(path_ext) if i.endswith('.TIF')]
        
                #for k in TIF_lis:
                 #   reproject_raster(path_ext+"\\"+k, path_ext+"\\GHS_builtup_"+str(i)+"_reprojected_4326"+k.split('_')[-1].split('.')[0]+'.tif')
            
                tif_lis = [i for i in os.listdir(path_ext) if i.endswith('.tif')]
        
                sum_lis = []

                for shp in shp_lis:
                    for ras in tif_lis:
                        sum_lis.append(get_clipped_density(shp, path_ext+"\\"+ras))
                sum_dic[i] = sum_lis
                
    return sum_dic

In [23]:
di = main('Dhaka')

C:\Users\wb542830\AppData\Roaming\Python\Python36\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\wb542830\AppData\Roaming\Python\Python36\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster

C:\Users\wb542830\.conda\envs\osm1\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster and polygon.
No Overlap between raster

In [67]:
di2 = {}
for i in di.keys():
    di2[i] = [round(i, 2) for i in di[i] if (i) and (np.isnan(i) != True)]

In [68]:
di2

{1990: [66.69, 71.59, 73.34, 71.31],
 2000: [71.69, 77.17, 79.61, 77.54],
 2014: [71.74, 77.35, 80.36, 78.97]}

In [66]:
#for i in di2.keys():
#    x = [500, 1000, 1500, 2000]
#    fig = px.line(x = x, y = di2[i], labels = {'x' : 'buffer distance (in m)', 'y' : 'share of built-up in {}'.format(i)}, 
#                  range_x = [250,2000], range_y = [0,100])
#    fig.show()

In [69]:
import plotly.graph_objects as go
import numpy as np
for i in di2.keys():
    x = [500, 1000, 1500, 2000]
    fig = go.Figure(data=go.Scatter(x=x, y=di2[i], mode='lines+markers', text=di2[i],  name='% Built-up'))

    fig.update_layout(
        title={
            'text': "Share of built-up in {} at different buffer levels".format(i),
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})

    fig.update_layout(xaxis_title='Buffer Distance (in m)',
                       yaxis_title='% Share of Built-up')
    fig.update_traces(hoverinfo='text+name', mode='lines+markers')
    fig.update_xaxes(range=[250,2100])
    fig.update_yaxes(range=[0,100])

    fig.show()